### Notebook to run RAG setup

In [9]:
import os
import getpass
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

import bs4 
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [10]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [12]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [13]:
vector_store = InMemoryVectorStore(embeddings)

In [14]:
# # Load and chunk contents of the blog
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

In [15]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://petguide.dk/hundefoder-maerker/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("entry-content single-page", "entry-title", "entry-meta uppercase is-xsmall")
        )
    ),
)
docs = loader.load()

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [ ]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [ ]:
prompt = ''

In [ ]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [ ]:
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [ ]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()